In [271]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

from core.pose_utils import Keypoints2Image, pad_keypoints

import cv2
from scipy.io import loadmat
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import numpy as np
import random
from core.segment import Segmentor
from copy import deepcopy
import math

In [67]:
HEIGHT = WIDTH = 256 # target resolution
kp2img = Keypoints2Image('openpose_body_25')

In [706]:
src_dir = '../data/img_highres/'
dst_dir = '../data/img_256/'
mask_dir = '../data/mask_256/'
multi_dst_dir = '../data/img_multi_256/'
multi_mask_dir = '../data/mask_multi_256/'

os.makedirs(multi_dst_dir, exist_ok=True)
os.makedirs(multi_mask_dir, exist_ok=True)

df = pd.read_pickle("../data/deepfashion_1.pickle")
df.drop(index=df[df.keypoints.isnull()].index, inplace=True)
#df.to_pickle("../data/deepfashion_1.pickle")


In [708]:
def get_empty_sample():
    image = 255*np.ones((HEIGHT, int(WIDTH*0.4),3), dtype=np.uint8)    
    return {'image':image, 
            'keypoints':None,
            'mask':image[:,:,0],
            'text':'', 
            'male':0, 
            'female':0}

In [710]:
def get_sample(sampled_idx,  angle=0., scale=1, crop_margin=0, mask_background=True):
    row = df.iloc[sampled_idx]
    keypoints = row.keypoints[0]
    image = cv2.imread(dst_dir+row.image)
    mask_file = (mask_dir+row.image).replace('.jpg','.bmp')
    mask = cv2.imread(mask_file, 0)

    ret = augment(image, keypoints, mask=mask, angle=angle, scale=scale, 
                   crop_margin=crop_margin, mask_background=mask_background)
    
    ret['text'] = row.text[0]
    ret['male'] = row.male
    ret['female'] = row.female
    ret['pose_score'] = row.pose_score
    return ret

def augment(image, keypoints, mask=None, scale=1, angle=0, crop_margin=0, mask_background=False):

    if mask_background and mask is not None:
        image = cv2.bitwise_and(image, image, mask=mask)        
        image[mask==0] = (255, 255, 255)
    
    height, width = image.shape[:2]
    center = (width//2, height//2)
    rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=angle, scale=scale)
    ty = int((1-scale)*height/2) if scale<=1 else 0
    tx = 0
    translation_matrix = np.array([[0, 0, tx],[0, 0, ty]])
    rotate_matrix += translation_matrix
    rotated_image = cv2.warpAffine(src=image, M=rotate_matrix, dsize=(width, height))
    
    if mask is not None:
        rotated_mask = cv2.warpAffine(src=mask, M=rotate_matrix, dsize=(width, height))
    else:
        rotated_mask = None

    # rotate keypoint
    kp_ = deepcopy(keypoints)
    kp_[:,2] = 1.
    center = (0.5, 0.5)
    rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=angle, scale=scale)
    ty = (1-scale)/2 if scale<=1 else 0
    tx = 0
    translation_matrix = np.array([[0, 0, tx],[0, 0, ty]])
    rotate_matrix += translation_matrix

    new_kp = np.dot(kp_, rotate_matrix.transpose())
    new_kp = np.concatenate((new_kp, np.expand_dims(keypoints[:,2].transpose(), 1)), axis=1)
    new_kp = new_kp.astype(np.float32)
    
    # crop
    left = 0
    right = width
    if mask is not None:
        vertical = np.mean(rotated_mask, axis=0)
        for w in range(width):
            if vertical[w] > 0.1:
                left = w
                break

        for w in range(width-1, -1, -1):
            if vertical[w] > 0.1:
                right = w
                break
            
        left = max(0, left-crop_margin)
        right = min(width, right+crop_margin)
        new_width = right - left
        ratio = width/new_width
        new_kp[:,0] = new_kp[:,0] - left/(width-1)
    
    new_kp[:,0] *= ratio
    return {'image':rotated_image[:,left:right], 
            'keypoints':new_kp,
            'mask':rotated_mask[:,left:right]}
'''
plt.figure(figsize=(10,10))
ret = get_sample(65718)
new_img = ret['image'][:,:,::-1]
h, w, _ = new_img.shape
kp2img = Keypoints2Image('openpose_body_25', (h,w))
new_kp_img = kp2img(np.expand_dims(ret['keypoints'],0))

#new_kp_img = new_kp_img[:h,:w,:]
merged = (new_kp_img*0.3 + 0.7*new_img)/255.
plt.imshow(merged)
plt.show()
plt.imshow(ret['mask'])
plt.show()

plt.figure(figsize=(10,10))
ret = augment(ret['image'], ret['keypoints'], mask=ret['mask'], scale=0.6)
new_img = ret['image'][:,:,::-1]
h, w, _ = new_img.shape
kp2img = Keypoints2Image('openpose_body_25', (h,w))
new_kp_img = kp2img(np.expand_dims(ret['keypoints'],0))
merged = (new_kp_img*0.3 + 0.7*new_img)/255.
plt.imshow(merged)
plt.show()
plt.imshow(ret['mask'])
'''

"\nplt.figure(figsize=(10,10))\nret = get_sample(65718)\nnew_img = ret['image'][:,:,::-1]\nh, w, _ = new_img.shape\nkp2img = Keypoints2Image('openpose_body_25', (h,w))\nnew_kp_img = kp2img(np.expand_dims(ret['keypoints'],0))\n\n#new_kp_img = new_kp_img[:h,:w,:]\nmerged = (new_kp_img*0.3 + 0.7*new_img)/255.\nplt.imshow(merged)\nplt.show()\nplt.imshow(ret['mask'])\nplt.show()\n\nplt.figure(figsize=(10,10))\nret = augment(ret['image'], ret['keypoints'], mask=ret['mask'], scale=0.6)\nnew_img = ret['image'][:,:,::-1]\nh, w, _ = new_img.shape\nkp2img = Keypoints2Image('openpose_body_25', (h,w))\nnew_kp_img = kp2img(np.expand_dims(ret['keypoints'],0))\nmerged = (new_kp_img*0.3 + 0.7*new_img)/255.\nplt.imshow(merged)\nplt.show()\nplt.imshow(ret['mask'])\n"

In [667]:
slots = []
'''
slots.append(get_sample(0))
slots.append(get_empty_sample())
slots.append(get_sample(2))
slots.append(get_sample(5))
slots.append(get_sample(4))
'''
slots.append(get_sample(65088, crop_margin=10))
slots.append(get_sample(39848, crop_margin=10))
slots.append(get_sample(21618, crop_margin=10))


In [711]:
def combine_slots(slots, crop_margin=0, space=0):
    male_count = 0
    female_count = 0
    text = ''
    merged = 255*np.ones((HEIGHT, WIDTH, 3), dtype=np.uint8)
    merged_mask = np.zeros((HEIGHT, WIDTH), dtype=np.uint8)
    
    n_slots = len(slots)
    slot_lens = [slot['image'].shape[1] for slot in slots]
    total_width = sum(slot_lens)
    total_width = max(WIDTH, total_width)
    scale_factor = WIDTH/total_width 
    concat_kps = []
    #space = 10 #pixels between people
    pose_scores = []
    start_x = 0
    for i, slot in enumerate(slots):
        slot_image = slot['image']
        kp = slot['keypoints']            
        mask = slot['mask']
        h, w, _ = slot_image.shape

        
        if kp is not None: # if not empty slot
            pose_scores.append(slot['pose_score'])
            ret = augment(slot_image, kp, mask=mask, scale=scale_factor, crop_margin=crop_margin)    
            slot_image = ret['image']
            mask = ret['mask']
            h, w, _ = slot_image.shape
            cropped_height = max(0, int(math.ceil((1-scale_factor)*(h-1)))+2)
            end_x = min(start_x + w, WIDTH-1)
            capped_w = end_x - start_x
            merged[cropped_height:, start_x:end_x] = slot_image[cropped_height:,:capped_w]
            merged_mask[cropped_height:, start_x:end_x] = mask[cropped_height:,:capped_w]

            kp = ret['keypoints']        
            ratio = w/WIDTH
            kp[:,0]= kp[:,0]*ratio +(start_x)/(WIDTH-1)
            concat_kps.append(deepcopy(kp))
            
            male_count+= slot['male']
            female_count+= slot['female']
            text += slot['text']
            #print("male", male_count, 'female',female_count)
        else:
            end_x = start_x + int(0.8*scale_factor*w)
        start_x = end_x + space
    concat_kps = np.array(concat_kps)

    return {'image':merged, 
            'keypoints':concat_kps,
            'mask':merged_mask,
            'text':text, 
            'male':male_count, 
            'female':female_count,
            'pose_score':pose_scores}
'''    
ret = combine_slots(slots, crop_margin=5)    
plt.figure(figsize=(10,10))
kp2img = Keypoints2Image('openpose_body_25')
kp_img = kp2img(ret['keypoints'])
image = ret['image']
plt.imshow((kp_img*0.3+0.7*image[:,:,::-1])/255.)
plt.show()
plt.figure(figsize=(10,10))
plt.imshow(ret['mask'])
'''

"    \nret = combine_slots(slots, crop_margin=5)    \nplt.figure(figsize=(10,10))\nkp2img = Keypoints2Image('openpose_body_25')\nkp_img = kp2img(ret['keypoints'])\nimage = ret['image']\nplt.imshow((kp_img*0.3+0.7*image[:,:,::-1])/255.)\nplt.show()\nplt.figure(figsize=(10,10))\nplt.imshow(ret['mask'])\n"

In [712]:
HEIGHT = 256
WIDTH = 256
multi_df = pd.DataFrame(columns=df.columns)

empty_percent = 0.0
male_percent = 0.2
random.seed(888)
np.random.seed(888)
N = len(df)
male_indices = list(df[df.male>0].index)

num_samples = {2:50000, 3:50000} # samples per numbeer of people

for p, num_sample in num_samples.items():
    for idx in tqdm(range(num_sample)):
        num_slots = p
        empty = random.uniform(0,1) < empty_percent
        if empty: # have empty slot
            num_slots += 1

        slots_avail = [1 for _ in range(num_slots)]
        if empty:
            slots_avail[random.randrange(0,num_slots,1)] = 0
        '''
        print("num_slot", num_slots)
        print("empty_slot_id", empty_slot_id)
        print("slots_avail", slots_avail)
        '''
        merged = np.zeros((TARGET_DIM, TARGET_DIM, 3), dtype=np.uint8)
        slot_width = TARGET_DIM//num_slots

        slots = []
        slot_ids = []
        captions = ""
        male_count = 0
        female_count = 0
        crop_margin = 5
        for not_empty in slots_avail:
            if not not_empty:
                slots.append(get_empty_sample())
                continue
            fail = True
            while fail:
                # for certain percentage, sample from male only
                if random.uniform(0,1) < male_percent:
                    sampled_idx = random.sample(male_indices, 1)[0]
                    #print("sample male", sampled_idx)
                else:
                    sampled_idx = random.randrange(N)
                sample = get_sample(sampled_idx, crop_margin=crop_margin+5)
                # only use picture that is show entire head
                if np.sum(sample['mask'], axis=1)[0]==0:
                    fail = False
                else:
                    fail = True
                    
            #print(sampled_idx)
            slot_ids.append(str(sampled_idx))
            slots.append(sample)

        ret = combine_slots(slots, crop_margin, space=0)
        merged = ret['image']
        mask = ret['mask']
        '''
        plt.imshow(merged[:,:,::-1])
        plt.show()
        
        plt.imshow(ret['mask'])
        plt.show()
        '''
        fname = '_'.join(slot_ids)
        dst_file = os.path.join(multi_dst_dir, f'{fname}.png')        
        cv2.imwrite(dst_file, merged)
        mask_file = os.path.join(multi_mask_dir, f'{fname}.bmp')
        cv2.imwrite(mask_file, mask)
        new_row = {'image':dst_file.replace('../data/',''), 
                   'text':[ret['text']],
                   'keypoints':ret['keypoints'], 
                   'num_people':p, 
                   'female':ret['female'], 
                   'male':ret['male'],
                   'pose_score':ret['pose_score']}
        multi_df = multi_df.append(new_row, ignore_index=True)


100%|█████████████████████████████████████| 50000/50000 [41:08<00:00, 20.25it/s]


In [716]:
multi_df

,image,text,num_people,keypoints,pose_score,female,male
0,img_multi_256/65088_39848.png,[the lady wore a white sleeveless dress.the la...,2,"[[[0.16590784, 0.11919098, 0.90167606], [0.171...","[[0.65654844], [0.53469163]]",2.0,0.0
1,img_multi_256/21618_60094.png,[the lady was wearing a red sleeveless tank.th...,2,"[[[0.122911714, 0.36436075, 0.90818834], [0.10...","[[0.8420277], [0.7847611]]",2.0,0.0
2,img_multi_256/41274_6689.png,[the lady is wearing a gray long-sleeved jacke...,2,"[[[0.24023373, 0.20909551, 0.89905256], [0.256...","[[0.45582512], [0.7980377]]",1.0,1.0
3,img_multi_256/40138_44021.png,[the lady was wearing a white long-sleeved swe...,2,"[[[0.2378331, 0.20879744, 0.9183851], [0.22754...","[[0.5174534], [0.48361027]]",2.0,0.0
4,img_multi_256/70297_17664.png,[the lady wore a black long-sleeved dress.the ...,2,"[[[0.21651572, 0.19086367, 0.8681383], [0.2758...","[[0.40513298], [0.4845749]]",2.0,0.0
...,...,...,...,...,...,...,...
99995,img_multi_256/27596_35660_38342.png,[the lady wore a black sleeveless top.the lady...,3,"[[[0.16470154, 0.44366732, 0.86534953], [0.159...","[[0.49246684], [0.44571579], [0.33535275]]",3.0,0.0
99996,img_multi_256/37284_71119_3035.png,[the man wore gray long-sleeved cardigan.the l...,3,"[[[0.09391874, 0.5262225, 0.88526535], [0.1743...","[[0.38793764], [0.42934003], [0.4358484]]",2.0,1.0
99997,img_multi_256/2833_58087_17586.png,[the lady was wearing a white long-sleeved blo...,3,"[[[0.16071099, 0.35884577, 0.899058], [0.12201...","[[0.7516109], [0.5253461], [0.4569836]]",3.0,0.0
99998,img_multi_256/27519_19941_21977.png,[the lady is wearing a white long-sleeved tee....,3,"[[[0.124214366, 0.41919452, 0.8980003], [0.133...","[[0.76353186], [0.4450743], [0.4255491]]",2.0,1.0


In [719]:
df.image = df.image.map(lambda x: 'img_256/'+x)

In [721]:
df_123 = pd.concat([df, multi_df], ignore_index=True)
df_123.to_pickle("../data/deepfashion_123.pickle")

In [723]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_123, test_size=1000, random_state=388)
test_df.reset_index(inplace=True)
train_df.reset_index(inplace=True)
train_df.to_pickle('../data/deepfashion_123_train.pickle')
test_df.to_pickle('../data/deepfashion_123_test.pickle')